In [2]:
import tensorflow as tf

# Check if TensorFlow detects any GPUs
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"GPUs detected: {[gpu.name for gpu in gpus]}")
    print("TensorFlow is using the GPU(s).")
else:
    print("No GPUs detected. TensorFlow is running on the CPU.")

# Verify the device used for computation
@tf.function
def simple_operation():
    return tf.matmul([[1.0]], [[1.0]])

# Run a simple operation to check the device
with tf.device('/GPU:0' if gpus else '/CPU:0'):
    result = simple_operation()
    print(f"Computation performed on: {result.device}")


GPUs detected: ['/physical_device:GPU:0']
TensorFlow is using the GPU(s).
Computation performed on: /job:localhost/replica:0/task:0/device:GPU:0


In [3]:
# pip install scikit-learn

In [4]:
# import sklearn
# print(sklearn.__version__)

In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Cropping2D, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2

In [6]:
# Define image dimensions and input shape
IMG_HEIGHT, IMG_WIDTH = 224, 224  # Resize images to 224x224
INPUT_SHAPE = (IMG_HEIGHT, IMG_WIDTH, 3)  # 3 channels for RGB images

In [7]:
# Load pre-trained MobileNetV2 model without the top (fully connected) layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=INPUT_SHAPE)


In [8]:

# Freeze the convolutional layers of MobileNetV2
base_model.trainable = False

In [9]:
# Create a Sequential CNN model using the pre-trained base model
model = Sequential([
    # Remove cropping and just use the base model
    base_model,
    
    # Global average pooling to reduce dimensions of feature maps
    GlobalAveragePooling2D(),
    
    # Fully connected layer for learning complex patterns
    Dense(128, activation='relu'),
    Dropout(0.5),  # Dropout layer to prevent overfitting

    # Output layer: 4 neurons for 4 classes, softmax for probabilities
    Dense(4, activation='softmax')
])

In [10]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),  # Adam optimizer
              loss='categorical_crossentropy',  # For multi-class classification
              metrics=['accuracy'])


In [11]:

# Print the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 128)               163968    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 4)                 516       
                                                                 
Total params: 2,422,468
Trainable params: 164,484
Non-tr

In [12]:

# Prepare the data using ImageDataGenerator
train_data_dir = "C:\\Users\\Vignesh\\Desktop\\Jupyter\\RoadConditionCNN\\Traning"
test_data_dir ="C:\\Users\\Vignesh\\Desktop\\Jupyter\\RoadConditionCNN\\Testing" 


In [13]:
# Apply data augmentation to training data
train_datagen = ImageDataGenerator(rescale=1.0/255,  # Normalize pixel values to [0, 1]
                                   rotation_range=15,  # Randomly rotate images
                                   width_shift_range=0.1,  # Randomly shift images horizontally
                                   height_shift_range=0.1,  # Randomly shift images vertically
                                   shear_range=0.1,  # Shear transformation
                                   zoom_range=0.1,  # Randomly zoom images
                                   horizontal_flip=True)  # Flip images horizontally


In [14]:
# For test data, only normalize pixel values
test_datagen = ImageDataGenerator(rescale=1.0/255)


In [15]:
# Load training data
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),  # Resize images
    batch_size=16,  # Number of images per batch
    class_mode='categorical'  # For multi-class classification
)

Found 1016 images belonging to 4 classes.


In [16]:
# Load test data
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=16,
    class_mode='categorical'
)

Found 256 images belonging to 4 classes.


In [ ]:
# Train the model
history = model.fit(
    train_generator,
    epochs=10,  # Number of training epochs
    validation_data=test_generator  # Use test data for validation
)

Epoch 1/10
64/64 [==============================] - 22s 335ms/step - loss: 0.1060 - accuracy: 0.9596 - val_loss: 1.6340 - val_accuracy: 0.7148
Epoch 2/10
64/64 [==============================] - 18s 279ms/step - loss: 0.1066 - accuracy: 0.9626 - val_loss: 0.9577 - val_accuracy: 0.6914
Epoch 3/10
64/64 [==============================] - 18s 278ms/step - loss: 0.0920 - accuracy: 0.9675 - val_loss: 1.7504 - val_accuracy: 0.7344
Epoch 4/10
64/64 [==============================] - 18s 285ms/step - loss: 0.0942 - accuracy: 0.9646 - val_loss: 1.3706 - val_accuracy: 0.7266
Epoch 5/10
64/64 [==============================] - 18s 282ms/step - loss: 0.1040 - accuracy: 0.9656 - val_loss: 1.3184 - val_accuracy: 0.7422
Epoch 6/10
64/64 [==============================] - 18s 280ms/step - loss: 0.0987 - accuracy: 0.9675 - val_loss: 1.2527 - val_accuracy: 0.7266
Epoch 7/10
64/64 [==============================] - 18s 282ms/step - loss: 0.0811 - accuracy: 0.9715 - val_loss: 1.1697 - val_accuracy: 0.7227

In [18]:
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

16/16 [==============================] - 2s 121ms/step - loss: 0.8763 - accuracy: 0.6758
Test Loss: 0.8763208389282227
Test Accuracy: 0.67578125


In [19]:
test_generator

In [20]:
# print(train_generator.class_indices)


In [21]:
# SAVE THE MODEL

# Save the model in HDF5 format
model.save('road_condition_model2.h5')